<a href="https://colab.research.google.com/github/luana-pgm/Quinto-Projeto-Analise-de-Dados-de-Sentimentos-de-filmes/blob/main/Naive_Bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Instalar PySpark
!pip -q install pyspark==3.5.1

# Montar o Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Caminho base (ajuste se o nome da pasta for diferente)
base_path = "/content/drive/MyDrive/Eixo_05 (1)/dados/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .appName("imdb-naive-bayes")
    .getOrCreate()
)

spark

In [ ]:
# ================================================================
# 3️⃣ Carregar os dados processados (TF-IDF)
# ================================================================
df = spark.read.parquet(base_path + "TFIDFfeaturizedData")

# Visualizar estrutura
df.printSchema()
df.show(5)
print(f"Total de registros: {df.count()}")

root
 |-- sentiment: string (nullable = true)
 |-- review: string (nullable = true)
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)

+---------+--------------------+-----+--------------------+
|sentiment|              review|label|            features|
+---------+--------------------+-----+--------------------+
| negative|i rented i am cur...|  0.0|(250,[3,13,16,24,...|
| negative|i am curious yell...|  0.0|(250,[1,3,6,11,12...|
| negative|if only to avoid ...|  0.0|(250,[3,26,61,77,...|
| negative|this film was pro...|  0.0|(250,[3,47,70,100...|
| negative|oh brotherafter h...|  0.0|(250,[3,5,10,46,4...|
+---------+--------------------+-----+--------------------+
only showing top 5 rows

Total de registros: 50000


In [ ]:
# ================================================================
# 4️⃣ Dividir em treino e teste
# ================================================================
train_df, test_df = df.randomSplit([0.8, 0.2], seed=42)
print(f"Treino: {train_df.count()} | Teste: {test_df.count()}")


Treino: 40099 | Teste: 9901


In [ ]:
# ================================================================
# 5️⃣ Modelo - Naive Bayes
# ================================================================
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Criar e treinar o modelo
nb = NaiveBayes(
    featuresCol="features",
    labelCol="label",
    modelType="multinomial"
)

nb_model = nb.fit(train_df)
print("✅ Modelo Naive Bayes treinado com sucesso!")

# Fazer previsões
pred_nb = nb_model.transform(test_df)
pred_nb.select("sentiment", "label", "prediction", "probability").show(10, truncate=False)

✅ Modelo Naive Bayes treinado com sucesso!
+---------+-----+----------+----------------------------------------+
|sentiment|label|prediction|probability                             |
+---------+-----+----------+----------------------------------------+
|negative |0.0  |0.0       |[0.5013840053863989,0.4986159946136011] |
|negative |0.0  |1.0       |[0.44690581945294533,0.5530941805470548]|
|negative |0.0  |0.0       |[0.5355761221606259,0.46442387783937406]|
|negative |0.0  |0.0       |[0.9420644904210138,0.05793550957898619]|
|negative |0.0  |0.0       |[0.9485888965706797,0.05141110342932045]|
|negative |0.0  |0.0       |[0.7749259795094423,0.22507402049055777]|
|negative |0.0  |0.0       |[0.6899263451292498,0.3100736548707502] |
|negative |0.0  |0.0       |[0.5745797510356832,0.42542024896431696]|
|negative |0.0  |0.0       |[0.8214258609969836,0.17857413900301658]|
|negative |0.0  |0.0       |[0.8976765272445517,0.10232347275544826]|
+---------+-----+----------+-------------------

In [ ]:
# ================================================================
# 6️⃣.1 Tabela visual de resultados
# ================================================================
import pandas as pd
from IPython.display import display

# Criar dicionário com as métricas
resultados = {
    "Métrica": ["Acurácia", "F1-Score"],
    "Valor": [round(acc_nb, 4), round(f1_nb, 4)]
}

# Converter para DataFrame
tabela_resultados = pd.DataFrame(resultados)

# Exibir em formato de tabela bonita
print("📊 RESULTADOS DO MODELO NAIVE BAYES")
display(tabela_resultados.style.set_table_styles(
    [{
        'selector': 'th',
        'props': [('background-color', '#f2f2f2'),
                  ('font-weight', 'bold'),
                  ('text-align', 'center')]
    },
     {
        'selector': 'td',
        'props': [('text-align', 'center')]
    }]
).hide(axis='index'))

📊 RESULTADOS DO MODELO NAIVE BAYES


Métrica,Valor
Acurácia,0.693600
F1-Score,0.693500
